## Will it Fit?  
**A container problem solved.**

The containerstore.com has a large selection of containers, but does not provide a search option to specify the size of the container.  My cat clawed the wicker baskets that fit perfectly in my small dresser, so I am in need of replacement containers in a very specific size.  Then I will be able to avoid trashing the entire piece of furniture.

First I scraped the data from the container store website collecting the name, dimensions, and url for each piece.  In this notebook we will be able to search for containers within a given range of appropriate dimensions.


In [19]:
import scrapy
import json
from pprint import pprint

Start by scraping and storing the data.

In [20]:
!rm container.json
!scrapy crawl container -o container.json

with open('container.json') as data_file:    
    data = json.load(data_file)



2017-07-20 16:10:23 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: container)
2017-07-20 16:10:23 [scrapy.utils.log] INFO: Overridden settings: {'BOT_NAME': 'container', 'FEED_FORMAT': 'json', 'FEED_URI': 'container.json', 'LOG_LEVEL': 'INFO', 'NEWSPIDER_MODULE': 'container.spiders', 'ROBOTSTXT_OBEY': True, 'SPIDER_MODULES': ['container.spiders']}
2017-07-20 16:10:23 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2017-07-20 16:10:23 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddleware',
 'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downlo

In [21]:
for i in range(0,len(data)):

    if((data[i]['dimensions'])==[]):
        print('no dimensions given:')
        print (data[i]['url'])

pprint(len(data))


no dimensions given:
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-compact-plastic-bins-4-pack-with-grey-lids/12d?productId=11006520
no dimensions given:
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-compact-plastic-bins-4-pack-with-white-lids/12d?productId=11006211
no dimensions given:
https://www.containerstore.com/s/storage/decorative-bins-baskets/folding-wire-storage-basket/12d?productId=11000564
262


We scraped the Container Store website and found 210 items from the "decorative bins and baskets" section and 52 from the "plastic bins and baskets" section.  Above we print links to the three items do not have dimensions given.  (Scraped on July 20, 2017)

In [22]:
for i in range (15,17):
    pprint(data[i])

{'dimensions': ['12-3/8" sq. x 12-3/8" h'],
 'price': ['12.99'],
 'title': 'Bigso Grey Fabric Storage Cube',
 'url': 'https://www.containerstore.com/s/storage/decorative-bins-baskets/bigso-grey-fabric-storage-cube/12d?productId=11006336'}
{'dimensions': ['11-1/4" x 8" x 4-3/8" h', '15" x 8-3/4" x 5" h'],
 'price': ['6.99', '9.99'],
 'title': 'Grey Laguna Cotton Fabric Storage Bins',
 'url': 'https://www.containerstore.com/s/storage/decorative-bins-baskets/grey-laguna-cotton-fabric-storage-bins/12d?productId=11005643'}


Next we adjust the format of the dimensions to make them searchable. They will be listed as [length, width, height] all in inches. 

In [23]:
from fractions import Fraction
# loop through the container links
for j in range (0,len(data)):
    data[j]['new dimensions']=[]
    # many container links have containers available in several different sizes, so we roam through each one
    for k in range(0,len(data[j]['dimensions'])):
        mydim = data[j]['dimensions'][k].split('x')
        newdim = []
        # each dimension of the container needs to be converted to a float and added to 'new dimensions'
        for i in range (0,len(mydim)):
            mysplit = mydim[i].split('sq.')[0]
            if '-' in mysplit:
                this_split = mysplit.split('-')
                num =0
                for p in this_split:
                    num += float(Fraction(p.split('"')[0]))
            else:
                num = float(Fraction(mysplit.split('"')[0]))
            # if the entry notes that it is square or a diameter, then the dimension needs to be inserted in two spots            
            double=int(('sq.' in mydim[i]) or ('diam.' in mydim[i]))
            for _ in range(0,double+1):
                newdim.append(num)
        data[j]['new dimensions'].append(newdim)

In [24]:
for j in range (0,2):
    print('')
    pprint(data[j])
    print(data[j]['dimensions'])
    print(data[j]['new dimensions'])
    


{'dimensions': [],
 'new dimensions': [],
 'price': ['29.99'],
 'title': 'White Compact Plastic Bins 4-Pack with Grey Lids',
 'url': 'https://www.containerstore.com/s/storage/plastic-bins-baskets/white-compact-plastic-bins-4-pack-with-grey-lids/12d?productId=11006520'}
[]
[]

{'dimensions': ['10" sq. x 10" h', '12" sq. x 12" h'],
 'new dimensions': [[10.0, 10.0, 10.0], [12.0, 12.0, 12.0]],
 'price': ['15.99', '19.99'],
 'title': 'Ashcraft Storage Cubes with Handles',
 'url': 'https://www.containerstore.com/s/storage/decorative-bins-baskets/ashcraft-storage-cubes-with-handles/12d?productId=11005637'}
['10" sq. x 10" h', '12" sq. x 12" h']
[[10.0, 10.0, 10.0], [12.0, 12.0, 12.0]]


The will_it_fit function will be given 6 values, 2 for each dimension (a lower and upper limit).  It will return the possible containers.

In [25]:
def will_it_fit(f1a,f1b,f2a,f2b,f3a,f3b):

    possibles = []
    for j in range(0,len(data)):
        num_of_containers  = len(data[j]['new dimensions'])
        for k in range(0,num_of_containers):
            if (len(data[j]['new dimensions'][k])==3):
                
                if ((f1a<data[j]['new dimensions'][k][0]<f1b) and (f2a<data[j]['new dimensions'][k][1]<f2b) 
                    and (f3a<data[j]['new dimensions'][k][2]<f3b)):

                    possibles.append([data[j],k])

    return possibles
    

The container that I am looking for should be in the range of 6-8.25" x 9-12" x 5-7"

In [26]:
f1a = 6.
f1b = 8.25
f2a = 9.
f2b = 12.
f3a = 5.
f3b = 7.

# f1a = 12.
# f1b = 15.
# f2a = 12.
# f2b = 15.
# f3a = 12.
# f3b = 15.


possibles = will_it_fit(f1a,f1b,f2a,f2b,f3a,f3b)

print('There are',len(possibles), 'containers that will fit your space:')
print('')
for c in possibles:
    print('')
    print(c[0]['title'],c[0]['dimensions'][c[1]])
    print('$' + str(c[0]['price'][c[1]]))
    print(c[0]['url'])


    

There are 8 containers that will fit your space:


Small Rectangular Hogla Storage Bin with Handles 7" x 11" x 5-1/4" h
$8.99
https://www.containerstore.com/s/storage/decorative-bins-baskets/small-rectangular-hogla-storage-bin-with-handles/12d?productId=10035370

Mondrian Storage Boxes with Lids 7" x 10-5/8" x 5-3/8" h
$6.99
https://www.containerstore.com/s/storage/decorative-bins-baskets/mondrian-storage-boxes-with-lids/12d?productId=10036979

Rectangular Hogla Storage Bin with Lid 7" x 11" x 5-1/4" h
$10.99
https://www.containerstore.com/s/storage/decorative-bins-baskets/rectangular-hogla-storage-bin-with-lid/12d?productId=11003202

Clear Handled Storage Baskets 6-1/4" x 11" x 5-1/8" h
$3.99
https://www.containerstore.com/s/storage/plastic-bins-baskets/clear-handled-storage-baskets/12d?productId=10022155

White Handled Storage Baskets 6-1/4" x 11" x 5-1/8" h
$3.99
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-handled-storage-baskets/12d?productId=10036528

Black

In [27]:
print(data[25]['url'])
print(data[25]['dimensions'])
print(data[25]['new dimensions'])

print(data[26]['dimensions'])
print(data[26]['new dimensions'])
# print(data[27]['dimensions'])
# print(data[27]['new dimensions'])

https://www.containerstore.com/s/storage/decorative-bins-baskets/small-hogla-storage-basket-with-handles/12d?productId=11003201
['9" x 5-3/4" x 4" h']
[[9.0, 5.75, 4.0]]
['15" x 11" x 6-1/4" h']
[[15.0, 11.0, 6.25]]


Some of the dimensions give only one or two dimensions.  These are accessories to other storage containers so they are excluded from the will_it_fit function that searches for containers by dimension.

In [28]:
odd = []
for j in range(0,len(data)):
    num_of_containers  = len(data[j]['new dimensions'])
    for k in range(0,num_of_containers):
        if (len(data[j]['new dimensions'][k])!=3):

            odd.append([data[j],k])
    
print('These are some results that are 1 or 2 dimensional divider accessories to containers.  These are excluded from container searches:')    
print('')
for o in odd:
    print('')    
    print(o[0]['title'],o[0]['dimensions'][o[1]])
    print(o[0]['url'])

These are some results that are 1 or 2 dimensional divider accessories to containers.  These are excluded from container searches:


White Wide AkroBin 10-7/8"
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-wide-akrobin/12d?productId=11004565

White Wide AkroBin 18" x 19"
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-wide-akrobin/12d?productId=11004565

White Narrow AkroBins 18" x 19"
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-narrow-akrobins/12d?productId=11003416

White Narrow AkroBins 10-7/8"
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-narrow-akrobins/12d?productId=11003416

White Medium AkroBins 14-3/4"
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-medium-akrobins/12d?productId=11003417

White Medium AkroBins 10-7/8"
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-medium-akrobins/12d?productId=11003417

White Medium AkroBins 18" x 19"
https://www.con